In [1]:
import gpxpy
import geojson


In [2]:
gpx_file = open('abidjan/Gbaka___Gare_de_bassam_Gonzagville___A___ZB_2019_10_14_10_27_41.gpx', 'r')
gpx = gpxpy.parse(gpx_file)
gpx.length_2d()

15801.901312791804

In [3]:
pt_mesure_names = {"vide": 1, "mi-plein": 2, "plein": 3, "surchargé": 4}
arrets = [elem for elem in gpx.waypoints if elem.name in pt_mesure_names.keys()]
arrets_times = [elem.time for elem in arrets]
arrets_times

[datetime.datetime(2019, 10, 14, 10, 45, 38, tzinfo=SimpleTZ("Z")),
 datetime.datetime(2019, 10, 14, 11, 17, 34, tzinfo=SimpleTZ("Z")),
 datetime.datetime(2019, 10, 14, 11, 18, 52, tzinfo=SimpleTZ("Z")),
 datetime.datetime(2019, 10, 14, 11, 22, 46, tzinfo=SimpleTZ("Z")),
 datetime.datetime(2019, 10, 14, 11, 26, 16, tzinfo=SimpleTZ("Z")),
 datetime.datetime(2019, 10, 14, 11, 28, 6, tzinfo=SimpleTZ("Z")),
 datetime.datetime(2019, 10, 14, 11, 29, 11, tzinfo=SimpleTZ("Z")),
 datetime.datetime(2019, 10, 14, 11, 31, 31, tzinfo=SimpleTZ("Z")),
 datetime.datetime(2019, 10, 14, 11, 32, 25, tzinfo=SimpleTZ("Z"))]

In [4]:
trajet = []
for track in gpx.tracks:
    entre_deux_arrets = []
    for segment in track.segments:
        for point in segment.points:
            if point.time in arrets_times: #on coupe
                geom_entre_deux_arrets = geojson.LineString(entre_deux_arrets)
                remplissage_candidate = [elem.name for elem in arrets if elem.time == point.time]
                remplissage = pt_mesure_names[remplissage_candidate[0]]
                #print(remplissage)
                geom_feature = geojson.Feature(geometry=geom_entre_deux_arrets, properties={"remplissage": remplissage}) 
                trajet.append(geom_feature)
                
                entre_deux_arrets = []
            else :
                entre_deux_arrets.append([point.longitude, point.latitude])
            #print(point.latitude)

#trajet

In [5]:
feature_collection = geojson.FeatureCollection(trajet)

with open('serpent_charge.geojson', 'w') as f:
    geojson.dump(feature_collection, f)